<img src="NU.png" width ="300" height=300>
<h1 style="text-align:center">An-Najah National University</h1>
<h1 style="text-align:center">Artificial Intelligence</h1>
&nbsp;
<h1 style="text-align:center;color:blue"> Smart Agent Seeking for “Gem”</h1>
&nbsp;
<h1 style="text-align:center">By: Sabri Odeh</h1>
<h1 style="text-align:center">Supervisor: Dr. Hamed Abdelhaq</h1>
&nbsp;
<h1 style="text-align:center">19 April 2021</h1>

In [15]:
#import libraries
import numpy as np
import pandas as pd
import random

In [16]:
#Build the shape of the environment
def environment_shape(lw):
    #Create an array (2D) have the rewards for each state. 
    #The array contains 3 rows and n columns
    #each value is initialized to -100.
    rewards = np.full((environment_rows, environment_columns), -100.)
    
    #set the rewards for all paths cells (grey squares)
    pc = {} #store locations in a dictionary
    pc[1] = [i for i in range(1, environment_columns-1)] 
    row_index =1
    for column_index in pc[row_index]:
         rewards[row_index, column_index] = lw
            
    #set corners of the environment to 0 
    rewards[0, 0] = 0
    rewards[0, environment_columns-1] = 0
    rewards[2, 0] = 0
    rewards[2, environment_columns-1] = 0
    
    #set the reward for the squares goal to 100 
    #squares goal have random locations
    while(1):
        x= random.randrange(3)
        y=  random.sample(range(1, environment_columns-1), 1)
        if (x!=1 and y!=[i for i in range(1, environment_columns-1)]):
            break 
    rewards[x ,y] = 100 

    while(1):
        x= random.randrange(3)
        y=  random.sample(range(1, environment_columns-1), 1)
        if (x!=1 and y!=[i for i in range(1, environment_columns-1)]):
            break
    rewards[x , y] = 100
   
    return rewards

In [17]:
#define a function that determines if the specified location is a terminal state
def is_terminal_state(current_row_index, current_column_index):
  #if the reward for this location is lw value, then it is grey squares
  if rewards[current_row_index, current_column_index] == lw:
    return False
  else:
    return True

#define a function that will choose a random grey square location 
def get_starting_location():
  #get a random row and column index
    current_row_index = np.random.randint(environment_rows)
    current_column_index = np.random.randint(environment_columns)
  #continue choosing random row and column indexes until a grey square state is identified
    while is_terminal_state(current_row_index, current_column_index):
        current_row_index = np.random.randint(environment_rows)
        current_column_index = np.random.randint(environment_columns)
    return current_row_index, current_column_index

#define an epsilon greedy algorithm that will choose which action to take next (where to move next)
def get_next_action(current_row_index, current_column_index, epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
    if np.random.random() < epsilon:
        return np.argmax(q_values[current_row_index, current_column_index])
    else: #choose a random action
        return np.random.randint(4)

#define a function that will get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
    new_row_index = current_row_index
    new_column_index = current_column_index
    if actions[action_index] == 'north ' and current_row_index > 0:
        new_row_index -= 1
    elif actions[action_index] == 'right' and current_column_index < environment_columns - 1:
        new_column_index += 1
    elif actions[action_index] == 'south ' and current_row_index < environment_rows - 1:
        new_row_index += 1
    elif actions[action_index] == 'left' and current_column_index > 0:
        new_column_index -= 1
    return new_row_index, new_column_index

#Define a function that will get the shortest path between any location within the environment
def get_shortest_path(start_row_index, start_column_index):
  #return immediately if this is an invalid starting location
    if is_terminal_state(start_row_index, start_column_index):
        return []
    else: #if this is a 'legal' starting location
        current_row_index, current_column_index = start_row_index, start_column_index
        shortest_path = []
        shortest_path.append([current_row_index, current_column_index])
    #continue moving along the path until we reach the goal
    while not is_terminal_state(current_row_index, current_column_index):
      #get the best action to take
        action_index = get_next_action(current_row_index, current_column_index, 0.7)
      #move to the next location on the path, and add the new location to the list
        current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
        shortest_path.append([current_row_index, current_column_index])
    return shortest_path

In [18]:
def Training(discount_factor,learning_rate,epoch):

    epsilon = 0.7 #the percentage of time when we should take the best action (instead of a random action)

    for episode in range(epoch):
      #get the starting location for this episode
        row_index, column_index = get_starting_location()

      #continue taking actions (moving) until we reach a terminal state
        while not is_terminal_state(row_index, column_index):
            #choose which action to take (where to move next)
            action_index = get_next_action(row_index, column_index, epsilon)

            #perform the chosen action, and transition to the next state (move to the next location)
            old_row_index, old_column_index = row_index, column_index #store the old row and column indexes
            row_index, column_index = get_next_location(row_index, column_index, action_index)

            #receive the reward for moving to the new state, and calculate the temporal difference
            reward = rewards[row_index, column_index]
            old_q_value = q_values[old_row_index, old_column_index, action_index]
            temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

            #update the Q-value for the previous state and action pair
            new_q_value = old_q_value + (learning_rate * temporal_difference)
            q_values[old_row_index, old_column_index, action_index] = new_q_value
        
    return


In [22]:
environment_rows = 3
print('The width of the environment:')
environment_columns = int(input())
#Create an array (3D) to hold the current Q-values for each state and action pair: Q(s, a) 
#The value of each (state, action) pair is initialized to 0.
q_values = np.zeros((environment_rows, environment_columns, 4))

#define actions
#numeric action codes: 0 = north, 1 = right, 2 = south, 3 = left
actions = ['north ', 'right', 'south ', 'left']

print('###########################################################')
print('The Living rewards is:')
lw = int(input())
rewards=environment_shape(lw)
#print rewards matrix
for row in rewards:
    print(row)
print('###########################################################')

#enter training parameters
print('The discount_factor is:')
discount_factor = float(input())

print('The learning_rate is:')
learning_rate = float(input())

print('The number of episodes is:')
epoch = int(input())

Training(discount_factor,learning_rate,epoch)
print('Training complete!')



The width of the environment:
6
###########################################################
The Living rewards is:
-1
[   0. -100. -100.  100. -100.    0.]
[-100.   -1.   -1.   -1.   -1. -100.]
[   0. -100. -100. -100.  100.    0.]
###########################################################
The discount_factor is:
0.9
The learning_rate is:
0.9
The number of episodes is:
1000
Training complete!


In [23]:
#display a Q_value Tabel
df = pd.DataFrame(q_values[1], columns = ['north','right','south','left'])#index => 0=S0, 1= S1, 2= S2 .....
df

,north,right,south,left
0,0.0,0.0,0.0,0.00
1,-100.0,79.1,-100.0,-100.00
2,-100.0,89.0,-100.0,70.19
3,100.0,89.0,-100.0,79.10
4,-100.0,-100.0,100.0,89.00
5,0.0,0.0,0.0,0.00


In [26]:
#display a few shortest paths
print(get_shortest_path(1, 2)) 

[[1, 2], [1, 3], [0, 3]]
